# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../WeatherPy/weather_df.csv")
weather_data

,Unnamed: 0,City,Latitude,Longitude,Max Temp.,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,okhotsk,59.3833,143.3000,9.86,85,100,25.97,RU,1612684428
1,1,faya,18.3851,42.4509,59.00,62,0,5.75,SA,1612684428
2,2,totness,5.8833,-56.3167,75.43,89,68,9.86,SR,1612684428
3,3,novikovo,51.1537,37.8879,3.25,89,36,10.38,RU,1612684429
4,4,vila velha,-20.3297,-40.2925,70.00,94,90,8.05,BR,1612684429
...,...,...,...,...,...,...,...,...,...,...
569,569,sabha,27.0377,14.4283,64.17,26,98,16.73,LY,1612684510
570,570,saint anthony,45.0205,-93.2180,-11.20,64,1,8.05,US,1612684510
571,571,gbadolite,4.2833,21.0167,83.07,16,0,6.15,CD,1612684510
572,572,nizhnekamsk,55.6366,51.8245,-4.00,71,90,17.90,RU,1612684510


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
#Configure gmaps
gmaps.configure(api_key=g_key)

#Store lat and lng as locations
locations = weather_data[["Latitude", "Longitude"]]

#Store humidity as weight
weights = weather_data["Humidity"]

In [5]:
#Plot Heatmap
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, dissipating=False,
                                max_intensity=100, point_radius = 5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
vacation_cities = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & 
                                  (weather_data["Max Temp."] >= 70) & (weather_data["Max Temp."] <= 80)].dropna()  
vacation_cities

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = vacation_cities.loc[:, ["City", "Country", "Latitude", "Longitude"]]
hotel_df["Hotel Name"] = ""

hotel_df

In [ ]:
target_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"type": "hotel",
         "keyword": "hotel",
         "radius": 5000,
         "key": g_key}

In [ ]:
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    city = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city}.")
    response = requests.get(target_url, params=params).json()
    results = response["results"]
    
    try:
        print(f"Closest hotel in {city}: {results[0]['name']}")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print(f'Missing Result...skipping')
        
 

In [ ]:
hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)


fig.add_layer(markers)

# Display figure
fig